In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import datetime

df_stikhi = pd.read_csv('./101120_stikhicats.csv', index_col = 0)
df_manifest = pd.read_csv('./083020_manifestcats.csv', index_col = 0)
df_dnevnik = pd.read_csv('./072420_dnevnikcats.csv', index_col = 0)
df_roman = pd.read_csv('./101820_romancats.csv', index_col = 0)
df_gazet = pd.read_csv('./102920_gazetcats.csv', index_col = 0)

In [50]:
passw = '****'
engine = sqlalchemy.create_engine('mysql+pymysql://madkehl:' + passw + '@127.0.0.1/prozhito_orig', encoding = 'utf-8')
connection = engine.connect()
metadata = sqlalchemy.MetaData()
notes = sqlalchemy.Table('notes', metadata, autoload = True, autoload_with = engine)
persons = sqlalchemy.Table('persons', metadata, autoload = True, autoload_with = engine)
diary =sqlalchemy.Table('diary', metadata, autoload = True, autoload_with = engine)

def execute(query):
    ResultProxy = connection.execute(query)
    return ResultProxy

def retrieve_notes(id_num):
    '''
    SELECT COUNT(YEAR(date)) AS NEWYEAR from notes WHERE YEAR(date) in (1901, 1903, 1906) GROUP BY YEAR(date); 
    **for some reason the year column is less complete than date column
    '''
    trial = execute('SELECT * FROM notes WHERE id = '+ str(id_num) + ';')
    ResultSet = trial.fetchall()
    return ResultSet



def resultToBio(diarynum):
    trans = execute('SELECT person FROM diary WHERE id = ' + str(diarynum) + ';')
    trans = trans.fetchall()
    if len(trans) < 1:
        return([[None, None, None]])
    bio_info = execute('SELECT firstName, lastName, info FROM persons WHERE id = ' + str(trans[0][0]) + ';')
    ResultSet = bio_info.fetchall()
    koshka_list = []
    for n in ResultSet:
        koshka_list.append([n[0], n[1], n[2]])
    return koshka_list


def sample(df):
    dice = range(max(df['entry_type']))
    for_tatyana = []
    id_col = []
    for i in dice:
        temp = df[df['entry_type'] == i]
        res = temp['entry_id'].sample(n=3, random_state=7)
        res = res.reset_index(drop = True)
        for_tatyana.append(res)
        id_col.append(i)
        id_col.append(i)
        id_col.append(i)
    return(for_tatyana, id_col)
    

C:\Users\madke\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning:

(3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")



In [51]:
full_entries = []
ids, entry_types = sample(df_stikhi)
for i in ids:
    for n in i:
        full_entries.append(retrieve_notes(n))

In [52]:
entry_id = []
diarynum = []
text = []
year = []

for i in full_entries:
    for n in i:
        entry_id.append(n[0])
        diarynum.append(n[1])
        text.append(n[2])
        year.append(n[3].year)

In [53]:
bios = []
for i in diarynum:
    bios.append(resultToBio(i))

In [54]:
first_name = []
last_name = []
info = []

for i in bios:
    for n in i:
        first_name.append(n[0])
        last_name.append(n[1])
        info.append(n[2])
        

In [55]:
len(entry_types)

39

In [56]:

full_entries_df = pd.DataFrame({
    
    'entry_type': entry_types,
    'entry_id': entry_id,
    'diarynum': diarynum,
    'text': text,
    'year': year,
    'first_name': first_name,
    'last_name': last_name,
    'info': info
    
    
})


In [58]:
full_entries_df.to_csv('./110320_poetrysamples.csv')